In [1]:
import numpy as np
import pandas as pd

# Classe Stump

- Um Stump, ou Decision Stump, é um modelo de classificação fraco que consiste em ser basicamente uma Árvore de Decisão contendo apenas altura igual à 1 e 2 folhas. Com isso, conseguimos deduzir de onde veio esse nome, visto que Stump é uma tradução literal de tronco de árvore.<br><br>

- Modelei um Stump como uma classe para facilitar o uso do mesmo ao longo do algoritmo do AdaBoost. Nela, teremos 3 parâmetros onde 2 podem ser opcionais:
    - feature: representa sobre qual feature estaremos aplicando o Stump.
    - value: representa o valor daquela feature. No caso do dataset poderá ser x, o ou b.
    - pred: representa qual predição será dada caso o valor da feature seja value.<br><br>
    
- Por exemplo, ao instanciarmos um Stump(0, x, 1) todos os valores iguais à x da feature 0 serão classificados como da classe 1 e classificados como a classe -1 caso contrário.<br><br>

- Além das features podemos passar TRUE e FALSE, como strings, para um Stump. Nesse caso, as predições serão sempre 1 caso o Stump seja inicializado com TRUE e -1 caso seja inicializado com FALSE.

In [2]:
class Stump:
    """
        Implementation of Stump Model - weak classifier
        A Stump is a decision tree with 1 node and 2 leafs only
    """
    
    def __init__(self, feature, value=None, pred=None):
        """
            Constructor of class
            Needs a feature, value and prediction associated with that feature
            
            Feature can also be TRUE or FALSE, in that case value=None and pred=None by default
        """
        self._pred = pred
        self._value = value
        self._feature = feature
        
        
    def __str__(self):
        """
            For printing a Stump
        """
        return "(%s, %s, %s)" % (self._feature, self._value, self._pred)
        
    def predict(self, X):
        """
            Class prediction by model (feature, value, pred)
            Returns a numpy.ndarray with values -pred and pred
        """
        n_samples = X.shape[0]
        
        if self._feature == "TRUE": # all classes are 1
            return np.ones(n_samples).astype(int)
        
        if self._feature == "FALSE": # all classes are -1
            return (-1) * np.ones(n_samples).astype(int)
        
        preds = np.empty(n_samples).astype(int)
        for i in range(n_samples):
            if X[i, self._feature] == self._value:
                preds[i] = self._pred
            else:
                preds[i] = (-1) * self._pred
                
        return preds

# Classe AdaBoost
- AdaBoost é um modelo de aprendizagem de máquina da família ``ensemble``. Ele se baseia na técnica de Boosting utilizando Stumps como classificadores mais fracos. Através destes, o Boosting tenta minimizar o alto viés associado a cada modelo dando a resposta como uma combinação das respostas destes classificadores mais fracos. No caso, cada classificador carregará consigo um peso, indicando o quão influente ele será na resposta final. Seja $H(X)$ a predição do modelo forte, $\alpha_i$ os pesos e $h_i(X)$ a predição dos classificadores fracos, teremos que:

$$ H(X) = sign(\sum_i^n \alpha_i * h_i(X)) $$

In [3]:
class AdaBoost:
    """
        Implementation of AdaBoost ensemble model
        It uses Stump classes as weak classifiers
    """
    
    def __init__(self, stumps_table, debug=False):
        """
            Constructor of class
            Needs a premade Stumps Table to be used by the model
        """ 
        self._alphas  = None
        self._weights = None
        self._stumps  = None
        self._stumps_table = stumps_table
        
    def _pick_best_stump(self, X, y):
        """
            This function picks the Stump in the Stumps Table that best
            minimizes the error given sample weights.

            Returns the best Stump model, it's error and predictions
        """    
        best_error = np.inf
        best_model = None
        best_preds = None

        for stump in self._stumps_table:
            preds = stump.predict(X)
            error = self._weights[(preds != y)].sum()

            if error < best_error:
                best_model = stump
                best_error  = error
                best_preds = preds
                
        return best_model, best_error, best_preds
    
    def print_stumps(self):
        """
            Only a debug method to print Stumps chose by model
            with it's alpha values
        """
        n_stumps = len(self._stumps)
        for i in range(n_stumps):
            print("- Weak classifier %d:" % (i+1))
            print("  - Alpha = %.5f" % self._alphas[i])
            print("  - Stump =", self._stumps[i])
            print()
        
    def fit(self, X, y, n_iter):
        """
            Fits model to data
            y values must be either 1 or -1
            n_iter defines how many iterations the algorithm will execute
        """
        if set(y) != {-1, 1}:
            raise ValueError("y values must be either 1 or -1")
            
        if n_iter <= 0:
            raise ValueError("The number of iterations must be greater than 0")
            
        n_samples = X.shape[0]
        self._stumps  = np.empty(n_iter, dtype=Stump)
        self._alphas  = np.empty(n_iter)
        self._weights = np.ones(n_samples) / n_samples # in the beginning all weights are 1/n
        
        for i in range(n_iter):
            stump, error, preds = self._pick_best_stump(X, y)
            
            # Computing alpha value for that Stump 
            alpha = 0.5 * np.log((1 - error) / error)
            
            # Computing next weights and normalizing
            weights = self._weights * np.exp(-alpha * preds * y)
            weights /= weights.sum()
            
            # Saving current iteration values
            self._stumps[i] = stump
            self._alphas[i] = alpha
            self._weights = weights
            
    def predict(self, X):
        """
            Class prediction by model given by dot(alphas, stumps prediction)
            Returns a numpy.ndarray with values -1 and 1
        """
        preds = np.array([stump.predict(X) for stump in self._stumps])
        return np.sign(np.dot(self._alphas, preds))

# "Corretude" usando dados do slide
- Usaremos o dataset do "Vampire" utilizado em sala de aula para corrigir o modelo implementado.

In [4]:
df = pd.DataFrame({
    "Vampire":    ['Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'N', 'N'],
    "Evil":       ['N', 'Y', 'Y', 'Y', 'Y', 'N', 'N', 'N', 'Y', 'N'],
    "Transforms": ['Y', 'Y', 'N', 'N', 'N', 'Y', 'Y', 'Y', 'N', 'N'],
    "Sparkly":    ['Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'N', 'Y']
})

df

,Vampire,Evil,Transforms,Sparkly
0,Y,N,Y,Y
1,Y,Y,Y,Y
2,Y,Y,N,N
3,Y,Y,N,Y
4,Y,Y,N,Y
5,Y,N,Y,Y
6,Y,N,Y,Y
7,N,N,Y,N
8,N,Y,N,N
9,N,N,N,Y


In [5]:
X = df.drop("Vampire", axis=1).values
y = df["Vampire"].replace(['N', 'Y'], [-1, 1]).values

stumps_table = []
for feature in range(X.shape[1]):
    for value in ['N', 'Y']:
        for pred in [-1, 1]:
            stumps_table.append(Stump(feature, value, pred))
            
stumps_table.append(Stump("TRUE"))
stumps_table.append(Stump("FALSE"))

clf = AdaBoost(stumps_table)
clf.fit(X, y, n_iter=3)
clf.print_stumps()

- Weak classifier 1:
  - Alpha = 0.69315
  - Stump = (2, N, -1)

- Weak classifier 2:
  - Alpha = 0.54931
  - Stump = (0, N, -1)

- Weak classifier 3:
  - Alpha = 0.44365
  - Stump = (1, N, -1)



- Temos que os valores de alpha foram computados de forma correta, porém escolhemos outros Stumps que resultaram na mesma resposta.